In [32]:
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import time

In [33]:
learning_rate = 1e-3
batch_size = 64
epoches = 50

if torch.cuda.is_available():
    print("CUDA inside")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), # The order should be right
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
                     ])
train = ImageFolder('/home/kost14/DL_with_Pytorch/Chapter3/dogs-vs-cats/train', simple_transform)
valid = ImageFolder('/home/kost14/DL_with_Pytorch/Chapter3/dogs-vs-cats/valid', simple_transform)

In [35]:
trainLoader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=4)
validLoader = DataLoader(valid, batch_size=batch_size, shuffle=False, num_workers=4)

In [36]:
cfg = {
    'VGG11': [64,'M',128,'M',256,256,'M',512,512,'M',512,512,'M'],
    'VGG13': [64,64,'M',128,128,'M',256,256,'M',512,512,'M',512,512,'M'],
    'VGG16': [64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M'],
    'VGG19': [64,64,'M',128,128,'M',256,256,256,256,'M',512,512,512,512,'M',512,512,512,512,'M']
}

In [41]:
class VGG(nn.Module):
    def __init__(self, vggname):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vggname])
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 2),
        )

    def _make_layers(self, cfg):
        in_channels = 3
        layers = []
        for i in cfg:
            if i == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, i, kernel_size=3, padding=1),
                           nn.BatchNorm2d(i),
                           nn.ReLU(inplace=True)]
                in_channels = i
        layers += [nn.AdaptiveAvgPool2d((7, 7))]
        return nn.Sequential(*layers)
                
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [42]:
vgg = VGG('VGG16')
if torch.cuda.is_available():
    vgg.cuda()
# alexnet.load_state_dict(torch.load('params.pkl'))

In [43]:
criterian = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.parameters(), lr=learning_rate, momentum=0.9)

In [44]:
try:
    since = time.time()
    best_acc = 0.0
    best_params = vgg.state_dict()
    
    for epoch in range(epoches):
        sum_loss_t = 0.0
        sum_acc_t = 0.0
        sum_loss_v = 0.0
        sum_acc_v = 0.0
        since_epoch = time.time()
            
        for (img, label) in trainLoader:
            img = img.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            output = vgg(img)
            loss = criterian(output, label)
            # backward
            loss.backward()
            optimizer.step()

            sum_loss_t += loss.item()
            _, predict = torch.max(output, 1)
            correct_num = (predict==label).sum()
            sum_acc_t += correct_num.item()

        for (img, label) in validLoader:
            img = img.to(device)
            label = label.to(device)

            output = vgg(img)
            loss = criterian(output, label)

            sum_loss_v += loss.item()
            _, predict = torch.max(output, 1)
            correct_num = (predict==label).sum()
            sum_acc_v += correct_num.item()
            if sum_acc_v > best_acc:
                best_params = alexnet.state_dict()

        sum_loss_t /= len(train)
        sum_acc_t /= len(train)
        sum_loss_v /= len(valid)
        sum_acc_v /= len(valid)
        time_elapsed = time.time() - since_epoch
        print("[{:d}/{:d}] Training Loss: {:.5f}, Acc: {:.2%}, Time: {:.0f}m {:.0f}s".format(epoch+1, epoches, sum_loss_t, sum_acc_t, time_elapsed//60, time_elapsed%60))
        print("[{:d}/{:d}] Validate Loss: {:.5f}, Acc: {:.2%}\n".format(epoch+1, epoches, sum_loss_v, sum_acc_v))
        
        if (epoch+1) % 10 == 0:
            torch.save(best_params, 'params.pkl')
            torch.cuda.empty_cache()
            print("Saved the model and emptied cache")
        
except Exception as e:
    torch.save(best_params, 'params.pkl')
    torch.cuda.empty_cache()
    raise e
else:
    torch.cuda.empty_cache()
    alltime_elapsed = time.time()-since
    print("Train complete in {:.0f}m {:.0f}s".format(alltime_elapsed//60, alltime_elapsed%60))

OSError: [Errno 12] Cannot allocate memory